# **Hand Gesture Recognition Model**

**Step 1: Import Necessary Libraries**

In [ ]:
# Importing libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report


**Step 2; Download and Prepare the Dataset**

In [ ]:
# Unzip the dataset
import zipfile

with zipfile.ZipFile("hand_gesture_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("hand_gesture_data")


In [ ]:
# Define dataset path
gesture_path = "hand_gesture_data"

# Display sample images
def display_sample_images(path, labels):
    plt.figure(figsize=(15, 5))
    for i, label in enumerate(labels):
        img_path = os.path.join(path, label, os.listdir(os.path.join(path, label))[0])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, len(labels), i + 1)
        plt.imshow(img)
        plt.title(label)
        plt.axis("off")
    plt.show()

# Example: Displaying five gesture classes
gesture_labels = os.listdir(gesture_path)[:5]
display_sample_images(gesture_path, gesture_labels)


**Step 3: Preprocess the Dataset**

In [ ]:
# Parameters for preprocessing
IMG_SIZE = 64  # Image size for resizing

# Preprocess images
def preprocess_data(path):
    data = []
    labels = []
    class_names = os.listdir(path)  # Folder names as labels
    class_mapping = {class_name: i for i, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            data.append(img)
            labels.append(class_mapping[class_name])

    return np.array(data), np.array(labels), class_mapping

# Load and preprocess the dataset
X, y, class_mapping = preprocess_data(gesture_path)

# Normalize pixel values and expand dimensions for CNN input
X = X / 255.0
X = np.expand_dims(X, axis=-1)  # Add channel dimension

# Convert labels to categorical format
y = to_categorical(y, num_classes=len(class_mapping))

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")
print(f"Class mapping: {class_mapping}")


**Step 4: Build the CNN Model**

In [ ]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(class_mapping), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()


**Step 5: Train the Model**

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)

# Plot training history
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


**Step 6: Evaluate the Model**

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Predict and generate classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_mapping.keys()))


**Step 7: Visualize Predictions**

In [ ]:
# Visualize some predictions
def visualize_predictions(X, y_true, y_pred, class_mapping, num_images=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        img = X[i].reshape(IMG_SIZE, IMG_SIZE)
        true_label = list(class_mapping.keys())[np.argmax(y_true[i])]
        pred_label = list(class_mapping.keys())[y_pred[i]]

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img, cmap="gray")
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis("off")
    plt.show()

visualize_predictions(X_test, y_test, y_pred_classes, class_mapping)


**Step 8: Save the Model**

In [ ]:
# Save the model
model.save("hand_gesture_recognition_model.h5")
print("Model saved as 'hand_gesture_recognition_model.h5'.")